In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 47.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=72b6cd5f8e88ed640b625d90ba5cadc1595b8718b0306484f30af68b0a0942ec
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
session = SparkSession.builder.appName("Classification").getOrCreate()

In [ ]:
data = session.read.csv("default_of_credit_card_clients.csv", header = True, inferSchema = True)

In [ ]:
data.show()

+------+---+---+---+---+---+---+---+---+---+---+------+------+------+------+------+------+-----+-----+-----+-----+------+-----+---+
|    X1| X2| X3| X4| X5| X6| X7| X8| X9|X10|X11|   X12|   X13|   X14|   X15|   X16|   X17|  X18|  X19|  X20|  X21|   X22|  X23|  Y|
+------+---+---+---+---+---+---+---+---+---+---+------+------+------+------+------+------+-----+-----+-----+-----+------+-----+---+
| 20000|  2|  2|  1| 24|  2|  2| -1| -1| -2| -2|  3913|  3102|   689|     0|     0|     0|    0|  689|    0|    0|     0|    0|  1|
|120000|  2|  2|  2| 26| -1|  2|  0|  0|  0|  2|  2682|  1725|  2682|  3272|  3455|  3261|    0| 1000| 1000| 1000|     0| 2000|  1|
| 90000|  2|  2|  2| 34|  0|  0|  0|  0|  0|  0| 29239| 14027| 13559| 14331| 14948| 15549| 1518| 1500| 1000| 1000|  1000| 5000|  0|
| 50000|  2|  2|  1| 37|  0|  0|  0|  0|  0|  0| 46990| 48233| 49291| 28314| 28959| 29547| 2000| 2019| 1200| 1100|  1069| 1000|  0|
| 50000|  1|  2|  1| 57| -1|  0| -1|  0|  0|  0|  8617|  5670| 35835| 20940|

In [ ]:
from pyspark.ml.feature import VectorAssembler
# Pyspark ml models takes only one independent variable and one dependent varibale
#but, we have multiple independent variabales

assembler_obj = VectorAssembler(inputCols= ["X1", "X2", "X3", "X4", "X5", "X6", "X7", "X8", "X9", "X10", "X11", "X12", "X13", "X14", "X15", "X16", "X17", "X18", "X19", "X20", "X21", "X22", "X23"], outputCol = "allfeatures")

In [ ]:
from pyspark.ml.classification import LogisticRegression
#featuresCol means independent varibale and labelCol means dependent variable
model = LogisticRegression(featuresCol= "allfeatures", labelCol = "Y") 

### Create Pipeline

In [ ]:
from pyspark.ml import Pipeline
mypipeline = Pipeline(stages = [assembler_obj, model])


In [ ]:
#splitting the dataset into training and test
training, test = data.randomSplit([0.7, 0.3])

In [ ]:
#training the model
fitmodel = mypipeline.fit(training)

In [ ]:
# Predict the values of the test dataset
result = fitmodel.transform(test)

In [ ]:
result.show(4)

+-----+---+---+---+---+---+---+---+---+---+---+-----+----+----+-----+-----+-----+----+----+----+----+----+---+---+--------------------+--------------------+--------------------+----------+
|   X1| X2| X3| X4| X5| X6| X7| X8| X9|X10|X11|  X12| X13| X14|  X15|  X16|  X17| X18| X19| X20| X21| X22|X23|  Y|         allfeatures|       rawPrediction|         probability|prediction|
+-----+---+---+---+---+---+---+---+---+---+---+-----+----+----+-----+-----+-----+----+----+----+----+----+---+---+--------------------+--------------------+--------------------+----------+
|10000|  1|  1|  1| 41|  0|  0|  3|  2|  2|  2| 6859|9751|9451| 9169|10056| 8559|3000|   0|   0|1002|   0|500|  0|[10000.0,1.0,1.0,...|[0.43976581686668...|[0.60820322817918...|       0.0|
|10000|  1|  1|  1| 50|  0|  0|  0|  0|  0|  0|10043|9616|9718| 9916| 9250| 7461|1194|1076| 274| 370| 472|375|  0|[10000.0,1.0,1.0,...|[0.73379895056942...|[0.67563837189168...|       0.0|
|10000|  1|  1|  2| 22|  0|  0| -1| -1| -1| -1| 2946|  

In [ ]:
#step1 :- Split
#step2 :- create model
#step3 :- traing model
#step4 :- Predict model
#step5 :- Evaluating the model

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
eval_obj = BinaryClassificationEvaluator(rawPredictionCol = 'probability', labelCol = "Y")
accuracy = eval_obj.evaluate(result)
print(accuracy)

0.7243794190739842
